In [1]:
# valuation_date時におけるcodeで指定した銘柄の個別銘柄分析
# SBI証券のモバイルの銘柄分析を参考に作成

In [2]:
### パラメータ設定セル
from datetime import date
from dateutil.relativedelta import relativedelta

# パラメータをセット
# code: 証券コード
# valuation_date: 過去にさかのぼって評価日をdatetime.dateでセット可能。デフォルト値は今日(date.today())
code = 4502
valuation_date = date.today()
chart_duration = relativedelta(days=90)
# valuation_date = date(2023, 11, 29)

In [3]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, FinancequotePl, KessanPl, KessanFig, PricelistFig, read_data, get_df_latest_yearly_performance, get_latest_stockprice, print_finance_quote
from lib_dataprocess import get_fig_expected_performance_progress_rate_pycharts, MeigaralistPl, get_fig_actual_performance_progress_rate_pycharts, ShikihoPl
import polars as pl

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

# dataの読み込み
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RevPPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)

In [4]:
### 最新情報
# 指定したcodeの指定した日における株価と各種ファンダメンタルズデータをまとめて標準出力する
# pricelist_dfは、raw_pricelistかreviced_pricelistかケースに応じて使い分ける。
# def print_finance_quote(
#        pricelist_df: pl.DataFrame,
#        finance_quote_df: pl.DataFrame,
#        code: int, 
#        valuation_date: date=date.today()
#    ) -> None:
print_finance_quote(PPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4)

武田薬品工業(4502)の銘柄情報

終値: 4176.0円(2024年12月03日)
予想配当利回り: 4.69%(2024年12月03日)
予想PER: 96.78倍(2024年12月03日)
実績PBR: 0.96倍(2024年12月03日)
自己資本比率: 48.1%(2024年12月03日)
予想ROE: 0.99%(2024年12月03日)
予想ROA: 0.48%(2024年12月03日)

4502(武田薬品工業)の四季報データ履歴

発行日: 2024年09月13日
【増益幅拡大】
  柱の潰瘍性大腸炎薬伸びる。約1400億円の構造改革費計上の一方、前期の肺がん治療薬など減損減る。好採算のADHD薬は後発品侵食が想定より後ろ倒しとなり営業益増額。税負担平常化。
【がん】
  中国アセンテージファーマ社と慢性骨髄性白血病薬の独占的ライセンス獲得のためのオプション契約を締結。米国に続き国内も営業・開発組織の人員削減へ。

発行日: 2024年06月14日
【復調】
  米国ADHD薬などの特許切れ影響大だが柱の潰瘍性大腸炎薬伸びる。約1400億円の構造改革費計上の一方、肺がん治療薬など減損減る。為替も追い風で営業益下げ止まる。税負担平常化。増配。
【費用】
  機動性向上のため米国中心に構造改革を実施、人員削減費用など計上。スイスのバイオ企業・ACイミューン社とアルツハイマー病薬候補のライセンス契約。

発行日: 2024年03月16日
【改善】
  コロナワクチン収入減る。ADHD薬『ビバンセ』特許切れも痛手。大幅営業減益。増配。25年3月期は『ビバンセ』の数量減重い。が、肺がん治療薬など1158億円の前期減損減り、反転増。税平常化。
【進展】
  大型化見込む睡眠障害治療薬候補『TAK―861』は第3相試験を24年秋までに開始予定。乾癬向け『TAK―279』は第3相向け試験の患者登録進行中。

発行日: 2023年12月15日
【大幅減額】
  好採算のADHD薬『ビバンセ』が後発品参入により後退。コロナワクチン収入なくなる。肺がん治療薬などで1158億円の減損計上。前号比営業減益幅拡大。過去税務評価の和解で税金減額。増配。25年3月期は巨額減損がなく上向き。
【加勢】
  23年10月米国で潰瘍性大腸炎薬の皮下注射剤発売。23年11月先天性血栓性

In [5]:
### 業績推移

# KessanFig(
#    code: int, 
#    settlement_type: Literal["通期", "四半期"], 
#    output_target: str = "jupyter",
#    start_settlement_date: date = date(1900, 1, 1),
#    end_settlement_date: date = date(2999, 12, 31)
#)

KFIG = KessanFig(code, "通期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [6]:
### 通期業績、業績予想
# def get_df_latest_yearly_performance(code: int, valuation_date: date=date.today()) -> pl.DataFrame:
df = get_df_latest_yearly_performance(code)
df

武田薬品工業(4502)の通期決算(評価日：2024年12月05日)


決算期,決算発表日,売上高,営業利益,経常利益,純利益,EPS,1株配当
str,date,i64,i64,i64,i64,f64,f64
"""2025年3月期(予)""",2024-10-31,4480000,265000,93000,68000,42.9,196.0
"""2024年3月期""",2024-05-09,4263762,214075,52791,144067,92.1,188.0


In [7]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [8]:
# codeで指定した銘柄のevaluation_dateで指定した時点での最新の年度決算予想に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# from plotly.graph_objects import Figure
# def get_fig_expected_performance_progress_rate_pycharts(code: int, evaluation_date: date=date.today()) -> Figure:

fig1 = get_fig_expected_performance_progress_rate_pycharts(code, valuation_date)
fig1.show()

武田薬品工業(4502)の2025年3月期第2四半期決算進捗率(評価日：2024-12-05)


In [9]:
last_year_valuation_date = date(valuation_date.year-1, valuation_date.month, valuation_date.day)

# codeで指定した銘柄のevaluation_dateで指定した時点での最新の四半期決算発表に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# valuation_dateで指定する日において、当年度の本決算が発表されていない日を指定した場合は前期末第4四半期の進捗率を表示するので、進捗率100%で表示される。
fig2 = get_fig_actual_performance_progress_rate_pycharts(code, last_year_valuation_date, KPL.df, MPL.df)
fig2.show()

武田薬品工業(4502)の2024年3月期第2四半期決算進捗率(評価日：2023-12-05)


In [10]:
# 株価推移
start_date = valuation_date - chart_duration
end_date = valuation_date + chart_duration
PFIG = PricelistFig(code, RevPPL.df, MPL.df, start_date, end_date)
PFIG.fig.show()